# Import Libraries

In [127]:
################ WEB SCRAPING MODULES ############
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import bs4
from fake_useragent import UserAgent
import requests
################ TIME MODLULES ###################
import time
from datetime import date 
import datetime
############## DATA MANIPULATION MODULES #########
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords

# Read 100 questions list

In [128]:
df = pd.read_excel('AnswersFromSource11_20.xlsx',engine='openpyxl')
link = 'https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism'
df[link]=np.nan
#df.drop(df.iloc[:, 1:25], inplace = True, axis = 1)
df

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
114,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN
115,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN
116,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN
117,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN


# Scrape QA pairs from website

In [129]:
# Open webpage in a new window
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism')



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/home/gopan/.wdm/drivers/chromedriver/linux64/92.0.4515.107/chromedriver] found in cache


In [130]:
# Parse text in webpage
source = driver.page_source
soup = bs4.BeautifulSoup(source, 'html.parser')

In [131]:
# Search the questions mentioned in webpage
l = []
for i in soup.find_all('div',{'class':'field-item even collapsed'}):
    print(i.text)
    l.append(i.text)

 What are Autism Spectrum Disorders?  
 What are the symptoms of Autism Spectrum Disorders? 
 What is Autistic Disorder? 
 What is Asperger's Syndrome? 
 What are the other ASDs (besides Autism and Asperger's Syndrome)? 
 What is Rett Syndrome? 
 Who Develops Autism Spectrum Disorders (ASDs)? 
 How Are ASDs Diagnosed? 
 What Are the Causes of Autism Spectrum Disorders (ASDs)? 
 How are ASDs treated? 


In [132]:
# Tried to search for answers of questions
k=[]
for div in soup.find_all('div',{'class':'field-item even'}):
    for p in div.find_all('p'):
        k.append(p.text)
for li in k:
    print(li)
    print("\n")
    print("\n")   

Below, find common questions about autism.




For the latest autism research, click here. Some autism research articles you might find useful:




Also, be sure to check out upcoming and past webinars about Autism, like this one:









Autism spectrum disorders (ASDs) represent a range of brain disorders that are characterized by restricted patterns of behavior and impairments in social communication and interactions. These disorders share common origins and features, but are classified as spectrum disorders because symptoms and severity vary among individuals.




Autism spectrum disorders (ASDs) represent a range of brain disorders that are characterized by restricted patterns of behavior and impairments in social communication and interactions. These disorders share common origins and features, but are classified as spectrum disorders because symptoms and severity vary among individuals.




Symptoms/behaviors of ASDs can range from mild to severe, and may seem to appear gradual

In [109]:
#find all bullet points 
ul = []
for ul in soup.find_all('ul'):
    for li in ul.find_all('li'):
        ul.append(li.text)
for li in ul:
    print(li)
    print("\n")
    print("\n")







<li>Limited pretend play</li>










<li>Lack of pointing to demonstrate interest</li>










<li>Reduced gaze following</li>










<li>Less frequent demonstration of repetitive, stereotypic behaviors</li>










<li>In children with autism between 2 years and 3 years of age, the following features may be observed:  </li>










<li>Communication difficulties</li>










<li>Socialization deficits with caregivers</li>










<li>Perceptual sensitivity</li>










<li>Other difficult behaviors</li>










Limited pretend play




Lack of pointing to demonstrate interest




Reduced gaze following




Less frequent demonstration of repetitive, stereotypic behaviors




In children with autism between 2 years and 3 years of age, the following features may be observed:  




Communication difficulties




Socialization deficits with caregivers




Perceptual sensitivity




Other difficult behaviors






# Check which questions are similar

In [133]:
stop_words = set(stopwords.words('english'))

def clean_text(sent):
    sent = sent.lower() # lowercase
    sent = re.sub(r'[^\w\s]', '', sent) # remove punctuations
    sent = re.sub('Autism Spectrum Disorder','ASD',sent) # Compress term
    sent = [w for w in sent.split() if not w.lower() in stop_words] # Remove stopwords
    sent = " ".join(sent)
    return sent

In [134]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return float(len(s1.intersection(s2)) / len(s1.union(s2)))

In [135]:
for i in l:
    i2 = clean_text(i)
    list1 = i2.split()
    
    temp1 = 0.0
    temp2 = ''
    temp3 = ''
    count = 0
    for z1, j in enumerate(df['Question']):
        j2 = clean_text(j)
        list2 = j2.split()
            
        sim = jaccard_similarity(list1, list2)
        
        if(sim>temp1):
            temp1 = sim
            temp2 = j
            temp3 = i
            count = z1+1
    if(temp1>=0.3):
        print('Website --> ',temp3)
        print('100 questions[ ',count,' ] list --> ',temp2)
        print(temp1)
        print('*'*100)

Website -->   What are Autism Spectrum Disorders?  
100 questions[  23  ] list -->  What are the Autism Spectrum Disorders (ASD)?
0.75
****************************************************************************************************
Website -->   What are the symptoms of Autism Spectrum Disorders? 
100 questions[  23  ] list -->  What are the Autism Spectrum Disorders (ASD)?
0.6
****************************************************************************************************
Website -->   What is Asperger's Syndrome? 
100 questions[  24  ] list -->  What is Asperger’s Syndrome?
1.0
****************************************************************************************************
Website -->   What are the other ASDs (besides Autism and Asperger's Syndrome)? 
100 questions[  25  ] list -->  How can you tell Autism from Asperger’s Syndrome?
0.5
****************************************************************************************************
Website -->   What is Rett Syndrome?

In [141]:
# Directly add answer to 'What are the Autism Spectrum Disorders (ASD)?' in existing dataframe

#df.iloc[0,0]
df[link].loc[22] = k[4]
df[link].loc[23] = k[9]
df[link].loc[26] = k[12]
#df['Question'].loc[23]
#k[12]
#df.iloc[1,1]=k[3]

/home/gopan/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [142]:
l

[' What are Autism Spectrum Disorders?  ',
 ' What are the symptoms of Autism Spectrum Disorders? ',
 ' What is Autistic Disorder? ',
 " What is Asperger's Syndrome? ",
 " What are the other ASDs (besides Autism and Asperger's Syndrome)? ",
 ' What is Rett Syndrome? ',
 ' Who Develops Autism Spectrum Disorders (ASDs)? ',
 ' How Are ASDs Diagnosed? ',
 ' What Are the Causes of Autism Spectrum Disorders (ASDs)? ',
 ' How are ASDs treated? ']

In [143]:
len(l)
l1=[l[2],l[4],l[6],l[7],l[8],l[9]]
l1

[' What is Autistic Disorder? ',
 " What are the other ASDs (besides Autism and Asperger's Syndrome)? ",
 ' Who Develops Autism Spectrum Disorders (ASDs)? ',
 ' How Are ASDs Diagnosed? ',
 ' What Are the Causes of Autism Spectrum Disorders (ASDs)? ',
 ' How are ASDs treated? ']

In [149]:
# manually select answers to questions in list 'l'

#l2 = [k[8]+'\n'+k[12]+'\n'+k[13]+'\n'+k[14],
#      k[16]+'\n'+k[17],
#     k[19]+'\n'+k[21]]
l2 =[k[7],k[10],k[13],k[15]+'\n'+k[16],k[18],k[20]+'\n'+k[21]]

print(l2[4])


Some combination of genetic, biological and environmental factors is believed to cause ASDs. Researchers are exploring several genes which are believed to contribute to the development of these disorders as well as several brain regions that have been linked to the disorders. Abnormal brain development during the first months of life is being studied to determine if structural abnormalities, such as in the mirror neuron systems, may be caused by genetic and/or environmental factors. Researchers are also exploring the effects of genetic imprinting in which a gene’s expression is determined by which parent donates the gene copy. Certain neurotransmitters, such as serotonin, dopamine, and epinephrine, may also function abnormally. In some cases, scientists are exploring the possibility that a faulty immune response to a virus, elevated concentrations of proteins in the blood at birth, dysregulation of specific neuropeptides or a major stress during pregnancy may lead to the disorder.



In [145]:
# Create new dataframe with QA pairs

df2 = pd.DataFrame(zip(l1,l2),columns=['Question',link])
df2

,Question,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism
0,What is Autistic Disorder?,"Autistic disorder, commonly referred to as aut..."
1,What are the other ASDs (besides Autism and A...,Other disorders in the spectrum include pervas...
2,Who Develops Autism Spectrum Disorders (ASDs)?,"ASDs are usually evident by the age of 3, thou..."
3,How Are ASDs Diagnosed?,"\nTo date, no biological diagnostic tests exis..."
4,What Are the Causes of Autism Spectrum Disord...,"\nSome combination of genetic, biological and ..."
5,How are ASDs treated?,"There is no one treatment for ASDs; however, i..."


In [146]:
# Concatenate existing and new dataframes
df3 = pd.concat([df,df2],axis=0)
df3 = df3.sort_values(by=list(df3.columns[1:])).reset_index(drop=True)
df3

,Question,https://springbrookbehavioral.com/frequently-asked-questions-about-autism/,https://www.aacap.org/AACAP/Families_and_Youth/Resource_Centers/Autism_Resource_Center/FAQ.aspx,https://www.autism.org.sg/living-with-autism/myths-and-facts-of-autism,https://www.autismsa.org.au/frequently-asked-questions-about-asd,https://www.bbrfoundation.org/faq/frequently-asked-questions-about-autism
0,What Can I Do To Prepare My Child For The Future?,Early intervention can do a lot toward fosteri...,NaN,NaN,NaN,NaN
1,Did I Cause My Child’s Autism?,"No. After getting an ASD diagnosis, this is th...",NaN,NaN,NaN,NaN
2,Can My Child’s Autism Be Cured?,No. Autism is not a disease that can be cured—...,NaN,NaN,NaN,NaN
3,How Can I Deal With This Diagnosis?,"Start by being patient and kind with yourself,...",NaN,NaN,NaN,NaN
4,How Do I Find Help For My Child?,Start by making sure that you and your child h...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
120,What are some ways that parents can reduce the...,NaN,NaN,NaN,NaN,NaN
121,Do some families deal with stress better than ...,NaN,NaN,NaN,NaN,NaN
122,Do siblings suffer increased stress as a resul...,NaN,NaN,NaN,NaN,NaN
123,What can I do about my children’s stress?,NaN,NaN,NaN,NaN,NaN


# Save Output

In [147]:
df3.to_excel('AnswersFromSource11_20.xlsx',index=False)